In [1]:
import inro
import numpy as np
import pandas as pd

In [2]:
# this notebook is run from inside EMME environment. Therefore it picks up the emmebank and scenario from the project that user opened. 
modeller = inro.modeller.Modeller()

emmebank = modeller.emmebank

In [4]:
scenario = emmebank.scenario(12)
scen_emmebank = scenario.emmebank

In [5]:
matrices_name_id_map = {
    "AM_WLK_TRN_WLK_DEMAND": "6",
    "AM_PNR_TRN_WLK_DEMAND" : "7",
    "AM_WLK_TRN_PNR_DEMAND" : "8",
    "AM_KNR_TRN_WLK_DEMAND" : "9",
    "AM_WLK_TRN_KNR_DEMAND" : "10",
    "AM_WLK_TRN_WLK_TRIM": "155",
    "AM_PNR_TRN_WLK_TRIM" : "177",
    "AM_WLK_TRN_PNR_TRIM" : "199",
    "AM_KNR_TRN_WLK_TRIM" : "221",
    "AM_WLK_TRN_KNR_TRIM" : "243",
    "AM_WLK_TRN_WLK_IVT": "146",
    "AM_PNR_TRN_WLK_IVT" : "168",
    "AM_WLK_TRN_PNR_IVT" : "190",
    "AM_KNR_TRN_WLK_IVT" : "212",
    "AM_WLK_TRN_KNR_IVT" : "224",
}

In [6]:
matrix_calc = modeller.tool("inro.emme.matrix_calculation.matrix_calculator")


In [7]:
output_file_name = "D:/kulshresthaa/MTC/TM2.2.1.1/output_summaries/trimmed_demand_report_am.csv"

In [8]:
report = []
period = "AM"
class_names = ["WLK_TRN_WLK", "PNR_TRN_WLK", "WLK_TRN_PNR", "KNR_TRN_WLK", "WLK_TRN_KNR"]

In [9]:
for name in class_names: 
    skim_mat_name = "%s_%s_IVT" % (period, name)
    demand_mat_name = "%s_%s_DEMAND" % (period, name)
    trim_mat_name = "%s_%s_TRIM" % (period, name)
    
    demand_id = matrices_name_id_map[demand_mat_name]
    ivt_id = matrices_name_id_map[skim_mat_name]
    trim_id = matrices_name_id_map[trim_mat_name]
    
    matrix = emmebank.matrix(f'mf{demand_id}')
    prev_demand = matrix.get_numpy_data()
    
    print("Trimming Demand for", name, "... ")
    print("    Prev Demand =", np.sum(prev_demand))
    
    spec_list = [
    {  # initialize TRIM skim
        "type": "MATRIX_CALCULATION",
        "constraint": None,
        "result": f'mf{trim_id}',
        "expression": '0',
    },
    
    {  # matrix used for trimming demands, set value to 1 if IVT is not infinite
        "type": "MATRIX_CALCULATION",
        "constraint": {
            "by_value": {
                "od_values": f'mf{ivt_id}',
                "interval_min": 0,
                "interval_max": 9999999,
                "condition": "INCLUDE"}
        },
        "result": f'mf{trim_id}',
        "expression": '1',
    },
    
    {  # update demand tables
        "type": "MATRIX_CALCULATION",
        "constraint": None,
        "result": f'mf{demand_id}',
        "expression": f'(mf{trim_id} * mf{demand_id})',
    },
    ]

    matrix_calc(spec_list, scenario=scenario)
    
    matrix = emmebank.matrix(f'mf{demand_id}')
    current_demand = matrix.get_numpy_data()
    
    print("    Current Demand =", np.sum(current_demand))
        
    trimmed_demand = np.sum(prev_demand) - np.sum(current_demand)
    
    print("    Trimmed Demand =", trimmed_demand)
    
    report.append((name,trimmed_demand))


Trimming Demand for: WLK_TRN_WLK ... 
    Prev Demand = 281040.4
    Current Demand = 276198.6
    Trimmed Demand = 4841.8125
Trimming Demand for: PNR_TRN_WLK ... 
    Prev Demand = 53121.586
    Current Demand = 52774.56
    Trimmed Demand = 347.02734
Trimming Demand for: WLK_TRN_PNR ... 
    Prev Demand = 1037.2283
    Current Demand = 1024.8384
    Trimmed Demand = 12.389893
Trimming Demand for: KNR_TRN_WLK ... 
    Prev Demand = 37145.855
    Current Demand = 36810.797
    Trimmed Demand = 335.0586
Trimming Demand for: WLK_TRN_KNR ... 
    Prev Demand = 7373.4683
    Current Demand = 7295.744
    Trimmed Demand = 77.72412


In [10]:
report

[('WLK_TRN_WLK', 4841.8125),
 ('PNR_TRN_WLK', 347.02734),
 ('WLK_TRN_PNR', 12.389893),
 ('KNR_TRN_WLK', 335.0586),
 ('WLK_TRN_KNR', 77.72412)]

In [11]:
pd.DataFrame(report).to_csv(output_file_name, index=False, header=['class','trimmed_demand'])